# Advent of code - Day5

https://adventofcode.com/2023/day/3

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', True)

import re
from math import log10, ceil

# Part 1

In [2]:
with open("day5_input.txt") as f:
    lines = f.read().splitlines() 

In [3]:
almanac = {}
for line in lines[2:]:
    if "map" in line:
        map_name = line
    elif line == "":
        pass
    else:
        numbers = [int(x) for x in re.findall(r'\d+', line)]
        if map_name in almanac.keys():
            almanac[map_name].append(numbers)
        else:
            almanac[map_name] = [numbers]
    

In [4]:
def find_mapped(mapping, seed):
    mapped_seed = ""
    for line in mapping:
        dest_range_start = line[0]
        source_range_start = line[1]
        range_length = line[2]
        if seed in range(source_range_start, source_range_start+range_length):
            position = range(source_range_start, source_range_start+range_length).index(seed)
            mapped_seed =  range(dest_range_start, dest_range_start+range_length)[position]
        else:
            pass
    if mapped_seed == "":
        mapped_seed = seed
    return mapped_seed 


In [5]:
seeds = [int(x) for x in re.findall(r'\d+', lines[0])]

In [6]:
def find_location(almanac, number):
    for mapping_key, mapping_value in almanac.items():
        # print("checking", mapping_key, number)
        number = find_mapped(mapping_value, number)
        # print("New number for ", mapping_key, number)
        # print(" --- Final location", number)
    return number

In [7]:
locations = []
for seed in seeds:
    location = find_location(almanac, seed)
    # print("Mapping ", seed, "to", location)
    locations.append(location)

print(min(locations))

510109797


# Part 2

### Approx approach

In [26]:
new_seeds = []
output = True

for i in range(0,len(seeds),2):
    new_seeds.append([seeds[i], seeds[i+1]])

step_size = int(pow(10, ceil(log10(max(s for s in seeds) / 100000))))
search_vals = {(ss, ss + sl, s): find_location(almanac, s) for ss, sl in new_seeds for s in range(ss, ss + sl, step_size)}
rough_est = min(search_vals.items(), key = lambda x: x[1])

seed_range_start, seed_range_end, best_est = rough_est[0]

if output:
    print(f'Best estimate: {best_est} in seed range {seed_range_start} to {seed_range_end}')
    print(f'Step size: {step_size:<8d}, best estimate: {best_est:<10d} near loc {rough_est[1]}')

while step_size > 1:
    left_search  = max(best_est - step_size, seed_range_start)
    right_search = min(best_est + step_size, seed_range_end)
    print("Searching", left_search, right_search, step_size)
    step_size = step_size // 10
    search_vals = {s: find_location(almanac, s) for s in range(left_search, right_search, step_size)}
    best_est, best_loc = min(search_vals.items(), key = lambda x: x[1])

    if output:
        print(f'Step size: {step_size:<8d}, best estimate: {best_est:<10d} near loc {best_loc}')

best_loc

Best estimate: 1950584943 in seed range 1736484943 to 2643914129
Step size: 100000  , best estimate: 1950584943 near loc 9709725
Searching 1950484943 1950684943 100000
Step size: 10000   , best estimate: 1950504943 near loc 9629725
Searching 1950494943 1950514943 10000
Step size: 1000    , best estimate: 1950497943 near loc 9622725
Searching 1950496943 1950498943 1000
Step size: 100     , best estimate: 1950497843 near loc 9622625
Searching 1950497743 1950497943 100
Step size: 10      , best estimate: 1950497843 near loc 9622625
Searching 1950497833 1950497853 10
Step size: 1       , best estimate: 1950497840 near loc 9622622


9622622

### Range approach

In [12]:
# For each map :
# We take each range and split them on intersections with given map
# Then we store the result of the map.
# And again until there is no range remaining.
# That was way too long and probably the last time I c/c code here xd


seeds, *maps = open("day5_input.txt").read().split('\n\n')
seeds = [int(seed) for seed in seeds.split()[1:]]
maps = [[list(map(int, line.split())) for line in m.splitlines()[1:]] for m in maps]

locations = []
for i in range(0, len(seeds), 2):
    ranges = [[seeds[i], seeds[i + 1] + seeds[i]]]
    results = []
    for _map in maps:
        while ranges:
            start_range, end_range = ranges.pop()
            for target, start_map, r in _map:
                end_map = start_map + r
                offset = target - start_map
                if end_map <= start_range or end_range <= start_map:  # no overlap
                    continue
                if start_range < start_map:
                    ranges.append([start_range, start_map])
                    start_range = start_map
                if end_map < end_range:
                    ranges.append([end_map, end_range])
                    end_range = end_map
                results.append([start_range + offset, end_range + offset])
                break
            else:
                results.append([start_range, end_range])
        ranges = results
        results = []
    locations += ranges
print(min(loc[0] for loc in locations))

9622622
